<a href="https://colab.research.google.com/github/MARMSKBO/Appliances-energy-prediction-data/blob/master/QMAppV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTAS
Con las afirmacion de la columna A
1. Formule una pregunta de opción múltiple.
2. Muestre 3 opciones de respuesta
3. Marque con una X la respuesta correcta al inicio de la respuesta correcta, segun la afirmacion de la columna A.

Con la misma pregunta de la columna C
El formato de la pregunta debe ser multiple opcion
1. Genere dos respuestas incorrestas
2. Agregue la respuesta correcta de la columna D y al inicio del parrafo de la respuesta coloque una" X)" sin caracter despues de ")".

In [1]:
# @title
!pip install python-docx pandas openpyxl gspread gspread_dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.9 MB/s eta 0:00:00


In [4]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


QMAppV2

* En modo secuencial muestra respuesta correcta si selecciona la incorrecta
* Muestra contador de preguntas

In [5]:
# @title
import pandas as pd
import ipywidgets as widgets
import random
import time
from IPython.display import display, clear_output
from datetime import timedelta

# Load the Excel file with all sheets (question sets)
file_path = '/content/drive/MyDrive/Colab Notebooks/Easa/MBase.xlsx'
sheets = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary

# List of sheet names (question sets)
sheet_names = list(sheets.keys())

# Variables to track the current question, score, and timer
score = 0
selected_questions = []  # To store the randomly selected questions
total_questions = 8

# Timer related variables
start_time = None
end_time = None
timer_running = False
timer_duration = 0  # in seconds

# Dropdown to select which set of questions to use
set_selection_dropdown = widgets.Dropdown(
    options=sheet_names,  # List of sets (sheet names)
    description='Question Set:',
    disabled=False
)

# Widget to select the number of questions to answer (initially set to a default)
question_number_widget = widgets.IntSlider(
    value=200,  # Default number of questions
    min=1,
    max=200,  # Default range (will update once a set is selected)
    step=1,
    description='# Questions:',
    continuous_update=False
)

# Timer enable option
use_timer_widget = widgets.Checkbox(
    value=False,
    description="Use Timer"
)

# Timer duration selection
timer_duration_widget = widgets.IntSlider(
    value=5,  # Default duration is 5 minutes
    min=1,
    max=240,
    step=1,
    description="Timer (min):",
    continuous_update=False
)
timer_duration_widget.disabled = True  # Disable until "Use Timer" is checked

# Button to start the quiz with the selected number of questions
start_button = widgets.Button(description="Start Quiz")

# Toggle to choose between random or sequential question order
order_toggle = widgets.ToggleButton(
    value=True,
    description='Random Order',
    disabled=False
)

# Function to handle quiz set change
def on_set_change(change):
    global total_questions
    selected_set_name = change['new']
    selected_set = sheets[selected_set_name]

    # Update the total question count based on the selected set
    total_questions = len(selected_set)

    # Update the question number slider range based on the total number of questions
    question_number_widget.max = total_questions
    question_number_widget.value = min(question_number_widget.value, total_questions)  # Ensure the value does not exceed max

# Attach the event handler for set selection
set_selection_dropdown.observe(on_set_change, names='value')

# Function to start the quiz
def start_quiz(btn):
    global selected_questions, score, start_time, end_time, timer_running, timer_duration

    # Obtener el conjunto seleccionado
    selected_set_name = set_selection_dropdown.value
    selected_set = sheets[selected_set_name]

    # Limpiar filas vacías
    selected_set = selected_set.dropna(subset=['Question', 'Answer1', 'Answer2', 'Answer3', 'AnswerCorrect']).reset_index(drop=True)

    num_questions = question_number_widget.value

    # Validar número de preguntas seleccionadas
    if num_questions == 0 or len(selected_set) == 0:
        display(widgets.HTML("<h3 style='color: red;'>Error: No questions available or selected!</h3>"))
        return

    # Configuración del temporizador
    if use_timer_widget.value:
        timer_duration = timer_duration_widget.value * 60  # Convertir minutos a segundos
        start_time = time.time()
        timer_running = True
    else:
        start_time = None
        end_time = None
        timer_running = False

    # Seleccionar preguntas en modo aleatorio o secuencial
    if order_toggle.value:
        if num_questions > len(selected_set):
            display(widgets.HTML("<h3 style='color: red;'>Error: Not enough questions available in the set!</h3>"))
            return
        selected_questions = random.sample(range(len(selected_set)), num_questions)  # Orden aleatorio
    else:
        selected_questions = list(range(num_questions))  # Orden secuencial

    score = 0  # Reiniciar puntaje
    load_question(0, selected_set)  # Cargar la primera pregunta
    start_button.disabled = True  # Deshabilitar el botón de inicio después de empezar
    display(timer_widget)  # Mostrar el widget del temporizador si está habilitado



start_button.on_click(start_quiz)

# Timer widget display
timer_widget = widgets.Label(value="Time Left: 00:00")

# Update the timer widget
def update_timer():
    if timer_running and start_time:
        elapsed_time = time.time() - start_time
        time_left = max(timer_duration - elapsed_time, 0)
        remaining_time = str(timedelta(seconds=int(time_left)))
        timer_widget.value = f"Time Left: {remaining_time.split()[2]}"
        if time_left == 0:
            end_time = time.time()
            timer_running = False
            show_results(time_out=True)

def load_question(current_index, selected_set):
    clear_output(wait=True)  # Clear previous output but keep widgets updated

    # Contador de pregunta (ejemplo: "1 of 10")
    question_counter = widgets.HTML(
        f"<h4>Question {current_index + 1} of {len(selected_questions)}</h4>"
    )

    # Display the question with a larger font size
    question_text = selected_set.loc[selected_questions[current_index], 'Question']
    question_widget = widgets.HTML(
        f"<h3 style='font-size: 20px; line-height: 1.5;'>{question_text}</h3>"
    )

    # Retrieve the answers
    answers = [
        selected_set.loc[selected_questions[current_index], 'Answer1'],
        selected_set.loc[selected_questions[current_index], 'Answer2'],
        selected_set.loc[selected_questions[current_index], 'Answer3']
    ]

    # Randomize the answer options
    correct_answer = selected_set.loc[selected_questions[current_index], 'AnswerCorrect']
    random.shuffle(answers)

    # Create radio buttons for answer options
    answer_options = widgets.RadioButtons(
        options=answers,
        disabled=False,
        layout=widgets.Layout(width='auto'),
        style={'description_width': 'initial'}
    )
    answer_options.add_class("custom-radio")  # Add a custom class for styling

    # Apply CSS styling
    display(widgets.HTML("""
        <style>
            .custom-radio label {
                font-size: 15px;  /* Adjust the font size as needed */
                display: block;
                margin-bottom: 15px;  /* Space between options */
            }
        </style>
    """))

    # Button to submit the answer
    submit_button = widgets.Button(
        description="Submit Answer",
        layout=widgets.Layout(width='150px')
    )

    # Submit button handler
    def on_submit(btn):
        submit_button.disabled = True  # Disable the button after the first click
        check_answer(answer_options, current_index, selected_set, correct_answer, answers)

    submit_button.on_click(on_submit)

    # Arrange widgets
    question_and_answers = widgets.VBox([question_counter, question_widget, answer_options], layout=widgets.Layout(padding="40px"))
    display(question_and_answers)
    display(submit_button)


# Function to check the selected answer and provide feedback
def check_answer(answer_options, current_index, selected_set, correct_answer, answers):
    global score

    # Check if the selected answer is correct
    selected_answer = answer_options.value

    # Feedback widget
    if selected_answer == correct_answer:
        feedback = widgets.HTML(
            value="<h4 style='color: green;'>Correct!</h4>"
        )
        score += 1
        correct_answer_widget = None  # No mostrar la respuesta correcta si el usuario acertó
    else:
        feedback = widgets.HTML(
            value="<h4 style='color: red;'>Incorrect!</h4>"
        )
        correct_answer_widget = widgets.HTML(
            value=f"<p style='color: green;'>Correct answer: {correct_answer}</p>"
        )

    # Sequential mode: Show feedback and "Next" button
    if not order_toggle.value:
        next_button = widgets.Button(
            description="Next",
            layout=widgets.Layout(width='150px')
        )

        def on_next(btn):
            next_index = current_index + 1
            if next_index < len(selected_questions):
                load_question(next_index, selected_set)  # Load the next question
            else:
                show_results()

        next_button.on_click(on_next)

        # Display feedback and the next button
        if correct_answer_widget:
            display(widgets.VBox([feedback, correct_answer_widget, next_button]))
        else:
            display(widgets.VBox([feedback, next_button]))
    else:
        # Random mode: Skip feedback and go to the next question
        next_index = current_index + 1
        if next_index < len(selected_questions):
            load_question(next_index, selected_set)  # Load the next question
        else:
            show_results()

# Function to show final results
def show_results(time_out=False):
    clear_output(wait=True)

    if len(selected_questions) == 0:
        display(widgets.HTML("<h3>No questions were answered!</h3>"))
        return

    if time_out:
        display(widgets.HTML("<h3>Time Out!</h3>"))
    else:
        percentage = (score / len(selected_questions)) * 100
        display(widgets.HTML(f"<h3>Quiz Completed!</h3>"))
        display(widgets.HTML(f"<p>You got {score} out of {len(selected_questions)} correct.</p>"))
        display(widgets.HTML(f"<p>Your score: {percentage:.2f}%</p>"))

    # Mostrar el tiempo tomado si el temporizador estaba activo
    if timer_running:
        elapsed_time = time.time() - start_time
        display(widgets.HTML(f"<p>Time taken: {str(timedelta(seconds=int(elapsed_time)))}</p>"))


# Function to handle changes to the timer enable checkbox
def update_timer_state(change):
    if use_timer_widget.value:
        timer_duration_widget.disabled = False  # Enable the timer duration slider
    else:
        timer_duration_widget.disabled = True  # Disable the timer duration slider

use_timer_widget.observe(update_timer_state, names='value')

# Display widgets
display(set_selection_dropdown)
display(question_number_widget)
display(order_toggle)  # Display the order toggle button
display(use_timer_widget)
display(timer_duration_widget)
display(start_button)

# Periodically update the timer widget
def update_periodically():
    # Verificar si hay preguntas para procesar
    if len(selected_questions) > 0 and timer_running:
        update_timer()
        Timer(1, update_periodically).start()


# Start the periodic update of the timer
if use_timer_widget.value:
    update_periodically()

HTML(value='\n        <style>\n            .custom-radio label {\n                font-size: 15px;  /* Adjust …

Button(description='Submit Answer', layout=Layout(width='150px'), style=ButtonStyle())